In [4]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

In [20]:
source_data_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109.json'
dest_data_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-flows.json'

In [21]:
def read_intramarket_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    print(f'Loaded data from {file_path}')
    return  data

def write_data_json(json_data, path):
    json_string = json.dumps(json_data)
    f = open(path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote intermarket json data to  {path}')



In [22]:
data = read_intramarket_json(source_data_path)

Loaded data from /Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109.json


In [23]:
data['metro_areas'].keys()

dict_keys(['AkronOH', 'AlbanyGA', 'AlbuquerqueNM', 'AllentownPA', 'AshevilleNC', 'AtlantaGA', 'AugustaGA', 'AustinTX', 'BakersfieldCA', 'BaltimoreMD', 'BatonRougeLA', 'BirminghamAL', 'BoiseCityID', 'BostonMA', 'BridgeportCT', 'BuffaloNY', 'CapeCoralFL', 'CharlestonWV', 'CharlotteNC', 'ChattanoogaTN', 'ChicagoIL', 'CincinnatiOH', 'ClevelandTN', 'ColoradoSpringsCO', 'ColumbiaSC', 'ColumbusOH', 'DallasFortWorthTX', 'DaytonOH', 'DeltonaFL', 'DenverCO', 'DesMoinesIA', 'DetroitMI', 'DurhamNC', 'ElPasoTX', 'FayettevilleAR', 'FayettevilleNC', 'FresnoCA', 'GrandRapidsMI', 'GreensboroNC', 'GreenvilleNC', 'HarrisburgPA', 'HartfordCT', 'HoustonTX', 'HuntsvilleAL', 'IndianapolisIN', 'JacksonMS', 'JacksonvilleFL', 'KansasCityMO', 'KilleenTX', 'KnoxvilleTN', 'LafayetteIN', 'LakelandFL', 'LancasterPA', 'LansingMI', 'LasVegasNV', 'LexingtonKY', 'LittleRockAR', 'LosAngelesCA', 'LouisvilleKY', 'MadisonWI', 'McAllenTX', 'MemphisTN', 'MiamiFL', 'MilwaukeeWI', 'MinneapolisMN', 'ModestoCA', 'MyrtleBeachSC', 

In [24]:
data['metro_areas']['DallasFortWorthTX'].keys()

dict_keys(['AkronOH', 'AlbanyGA', 'AlbuquerqueNM', 'AllentownPA', 'AshevilleNC', 'AtlantaGA', 'AugustaGA', 'AustinTX', 'BakersfieldCA', 'BaltimoreMD', 'BatonRougeLA', 'BirminghamAL', 'BoiseCityID', 'BostonMA', 'BridgeportCT', 'BuffaloNY', 'CapeCoralFL', 'CharlestonWV', 'CharlotteNC', 'ChattanoogaTN', 'ChicagoIL', 'CincinnatiOH', 'ClevelandTN', 'ColoradoSpringsCO', 'ColumbiaSC', 'ColumbusOH', 'DaytonOH', 'DeltonaFL', 'DenverCO', 'DesMoinesIA', 'DetroitMI', 'DurhamNC', 'ElPasoTX', 'FayettevilleAR', 'FayettevilleNC', 'FresnoCA', 'GrandRapidsMI', 'GreensboroNC', 'GreenvilleNC', 'HarrisburgPA', 'HartfordCT', 'HoustonTX', 'HuntsvilleAL', 'IndianapolisIN', 'JacksonMS', 'JacksonvilleFL', 'KansasCityMO', 'KilleenTX', 'KnoxvilleTN', 'LafayetteIN', 'LakelandFL', 'LancasterPA', 'LansingMI', 'LasVegasNV', 'LexingtonKY', 'LittleRockAR', 'LosAngelesCA', 'LouisvilleKY', 'MadisonWI', 'McAllenTX', 'MemphisTN', 'MiamiFL', 'MilwaukeeWI', 'MinneapolisMN', 'ModestoCA', 'MyrtleBeachSC', 'NashvilleTN', 'NewHa

In [25]:
flows = []
for source_metro in data['metro_areas'].keys():
    #print(source_metro)
    metro_data = data['metro_areas'][source_metro]
    for dest_metro in metro_data.keys():
        if dest_metro not in ['all_other_markets', 'all_markets', 'top_same_state_markets', 'other_market_same_state_markets', 'all_same_state_markets', 'all_other_markets_excl_same_state', 'all_markets_excl_same_state', 'top_120', 'top_50', 'top_20', 'top_25_list']:
            if source_metro == 'DallasFortWorthTX' or dest_metro == 'DallasFortWorthTX':
                #print(dest_metro)
                #print(metro_data[dest_metro])
                count = metro_data[dest_metro][0]
                flow = {
                    'origin': source_metro,
                    'dest': dest_metro,
                    'count': count
                }
                flows.append(flow)

flows

[{'origin': 'AkronOH', 'dest': 'DallasFortWorthTX', 'count': 182},
 {'origin': 'AlbanyGA', 'dest': 'DallasFortWorthTX', 'count': 172},
 {'origin': 'AlbuquerqueNM', 'dest': 'DallasFortWorthTX', 'count': 893},
 {'origin': 'AllentownPA', 'dest': 'DallasFortWorthTX', 'count': 174},
 {'origin': 'AshevilleNC', 'dest': 'DallasFortWorthTX', 'count': 467},
 {'origin': 'AtlantaGA', 'dest': 'DallasFortWorthTX', 'count': 4404},
 {'origin': 'AugustaGA', 'dest': 'DallasFortWorthTX', 'count': 291},
 {'origin': 'AustinTX', 'dest': 'DallasFortWorthTX', 'count': 21191},
 {'origin': 'BakersfieldCA', 'dest': 'DallasFortWorthTX', 'count': 110},
 {'origin': 'BaltimoreMD', 'dest': 'DallasFortWorthTX', 'count': 827},
 {'origin': 'BatonRougeLA', 'dest': 'DallasFortWorthTX', 'count': 479},
 {'origin': 'BirminghamAL', 'dest': 'DallasFortWorthTX', 'count': 568},
 {'origin': 'BoiseCityID', 'dest': 'DallasFortWorthTX', 'count': 513},
 {'origin': 'BostonMA', 'dest': 'DallasFortWorthTX', 'count': 1140},
 {'origin': '

In [26]:
write_data_json(flows,dest_data_path)

Wrote intermarket json data to  /Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-flows.json


In [27]:
market_mapping_settings_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/v5_12th_run Brian Stobie/Market_Mapping_Settings.csv'
dest_data_path = '/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-locations.json'

In [44]:
#     data_rows = []
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)

def read_market_settings(file_path):
    data_rows = []
    with open(file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    return  data_rows

def write_intermarket_locations_json(json_data, path):
    json_string = json.dumps(json_data)
    f = open(path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote intermarket locations json data to  {path}')


In [45]:
data = read_market_settings(market_mapping_settings_path)
data

[{'MarketName': 'AbileneTX',
  'Market_Display_Name': 'Abilene, TX',
  'StateFips': '',
  'CenterLat': '32.453549',
  'CenterLong': '-99.730981',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AkronOH',
  'Market_Display_Name': 'Akron, OH',
  'StateFips': '',
  'CenterLat': '41.080491',
  'CenterLong': '-81.517454',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AlbanyGA',
  'Market_Display_Name': 'Albany, GA',
  'StateFips': '',
  'CenterLat': '31.577248',
  'CenterLong': '-84.152334',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AlbanyOR',
  'Market_Display_Name': 'Albany, OR',
  'StateFips': '',
  'CenterLat': '44.635272',
  'CenterLong': '-123.10634',
  'GeoTypeGroup': '1 - Designated Metro Areas',
  'DefaultZoomLevel': '8'},
 {'MarketName': 'AlbanyNY',
  'Market_Display_Name': 'Albany-Schenectady-Troy, NY',
  'StateFips': '',
  'CenterLat': '4

In [46]:
data[0]

{'MarketName': 'AbileneTX',
 'Market_Display_Name': 'Abilene, TX',
 'StateFips': '',
 'CenterLat': '32.453549',
 'CenterLong': '-99.730981',
 'GeoTypeGroup': '1 - Designated Metro Areas',
 'DefaultZoomLevel': '8'}

In [47]:
locations = []
for location in data:
    if location['GeoTypeGroup'] == '1 - Designated Metro Areas':
        location = {
            "id": location['MarketName'],
            "name":location['Market_Display_Name'] ,
            "lat": location['CenterLat'],
            "lon": location['CenterLong']
        }
        locations.append(location)
locations

[{'id': 'AbileneTX',
  'name': 'Abilene, TX',
  'lat': '32.453549',
  'lon': '-99.730981'},
 {'id': 'AkronOH',
  'name': 'Akron, OH',
  'lat': '41.080491',
  'lon': '-81.517454'},
 {'id': 'AlbanyGA',
  'name': 'Albany, GA',
  'lat': '31.577248',
  'lon': '-84.152334'},
 {'id': 'AlbanyOR',
  'name': 'Albany, OR',
  'lat': '44.635272',
  'lon': '-123.10634'},
 {'id': 'AlbanyNY',
  'name': 'Albany-Schenectady-Troy, NY',
  'lat': '42.65164',
  'lon': '-73.754175'},
 {'id': 'AlbuquerqueNM',
  'name': 'Albuquerque, NM',
  'lat': '35.087707',
  'lon': '-106.651097'},
 {'id': 'AlexandriaLA',
  'name': 'Alexandria, LA',
  'lat': '31.311631',
  'lon': '-92.444484'},
 {'id': 'AllentownPA',
  'name': 'Allentown-Bethlehem-Easton, PA',
  'lat': '40.603692',
  'lon': '-75.467035'},
 {'id': 'AltoonaPA',
  'name': 'Altoona, PA',
  'lat': '40.516972',
  'lon': '-78.402261'},
 {'id': 'AmarilloTX',
  'name': 'Amarillo, TX',
  'lat': '35.207216',
  'lon': '-101.830982'},
 {'id': 'AmesIA', 'name': 'Ames, IA

In [48]:
write_intermarket_locations_json(locations,dest_data_path)

Wrote intermarket locations json data to  /Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-locations.json


In [49]:
locations

[{'id': 'AbileneTX',
  'name': 'Abilene, TX',
  'lat': '32.453549',
  'lon': '-99.730981'},
 {'id': 'AkronOH',
  'name': 'Akron, OH',
  'lat': '41.080491',
  'lon': '-81.517454'},
 {'id': 'AlbanyGA',
  'name': 'Albany, GA',
  'lat': '31.577248',
  'lon': '-84.152334'},
 {'id': 'AlbanyOR',
  'name': 'Albany, OR',
  'lat': '44.635272',
  'lon': '-123.10634'},
 {'id': 'AlbanyNY',
  'name': 'Albany-Schenectady-Troy, NY',
  'lat': '42.65164',
  'lon': '-73.754175'},
 {'id': 'AlbuquerqueNM',
  'name': 'Albuquerque, NM',
  'lat': '35.087707',
  'lon': '-106.651097'},
 {'id': 'AlexandriaLA',
  'name': 'Alexandria, LA',
  'lat': '31.311631',
  'lon': '-92.444484'},
 {'id': 'AllentownPA',
  'name': 'Allentown-Bethlehem-Easton, PA',
  'lat': '40.603692',
  'lon': '-75.467035'},
 {'id': 'AltoonaPA',
  'name': 'Altoona, PA',
  'lat': '40.516972',
  'lon': '-78.402261'},
 {'id': 'AmarilloTX',
  'name': 'Amarillo, TX',
  'lat': '35.207216',
  'lon': '-101.830982'},
 {'id': 'AmesIA', 'name': 'Ames, IA

In [50]:
loc_df = pd.DataFrame.from_dict(locations)
loc_df

,id,name,lat,lon
0,AbileneTX,"Abilene, TX",32.453549,-99.730981
1,AkronOH,"Akron, OH",41.080491,-81.517454
2,AlbanyGA,"Albany, GA",31.577248,-84.152334
3,AlbanyOR,"Albany, OR",44.635272,-123.10634
4,AlbanyNY,"Albany-Schenectady-Troy, NY",42.65164,-73.754175
...,...,...,...,...
376,YakimaWA,"Yakima, WA",46.60734,-120.488339
377,YorkPA,"York-Hanover, PA",39.96062,-76.726678
378,YoungstownOH,"Youngstown-Warren-Boardman, OH",39.96062,-76.726678
379,YubaCityCA,"Yuba City, CA",39.146297,-121.637621


In [51]:
loc_df.to_csv('/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-locations.csv', index=False)

In [36]:
flows

[{'origin': 'AkronOH', 'dest': 'DallasFortWorthTX', 'count': 182},
 {'origin': 'AlbanyGA', 'dest': 'DallasFortWorthTX', 'count': 172},
 {'origin': 'AlbuquerqueNM', 'dest': 'DallasFortWorthTX', 'count': 893},
 {'origin': 'AllentownPA', 'dest': 'DallasFortWorthTX', 'count': 174},
 {'origin': 'AshevilleNC', 'dest': 'DallasFortWorthTX', 'count': 467},
 {'origin': 'AtlantaGA', 'dest': 'DallasFortWorthTX', 'count': 4404},
 {'origin': 'AugustaGA', 'dest': 'DallasFortWorthTX', 'count': 291},
 {'origin': 'AustinTX', 'dest': 'DallasFortWorthTX', 'count': 21191},
 {'origin': 'BakersfieldCA', 'dest': 'DallasFortWorthTX', 'count': 110},
 {'origin': 'BaltimoreMD', 'dest': 'DallasFortWorthTX', 'count': 827},
 {'origin': 'BatonRougeLA', 'dest': 'DallasFortWorthTX', 'count': 479},
 {'origin': 'BirminghamAL', 'dest': 'DallasFortWorthTX', 'count': 568},
 {'origin': 'BoiseCityID', 'dest': 'DallasFortWorthTX', 'count': 513},
 {'origin': 'BostonMA', 'dest': 'DallasFortWorthTX', 'count': 1140},
 {'origin': '

In [37]:
flows_df = pd.DataFrame.from_dict(flows)
flows_df

,origin,dest,count
0,AkronOH,DallasFortWorthTX,182
1,AlbanyGA,DallasFortWorthTX,172
2,AlbuquerqueNM,DallasFortWorthTX,893
3,AllentownPA,DallasFortWorthTX,174
4,AshevilleNC,DallasFortWorthTX,467
...,...,...,...
233,WichitaKS,DallasFortWorthTX,723
234,WinstonNC,DallasFortWorthTX,248
235,WorcesterMA,DallasFortWorthTX,143
236,YorkPA,DallasFortWorthTX,88


In [39]:
flows_df.to_csv('/Users/alanmccann/Dropbox/bain/bain-uploads/intermarketdata-20221109-flows.csv', index=False)